In this notebook we will complete a small end to end data science tutorial that employs LakeFS-spec for data versioning.
We will use weather data to train a random forest classifier and aim to predict whether it is raining a day from now given the current weather. 

Lets set the environment up. You can create it with `python -m venv .demo-venv` in the command line. Then activate it (`source .demo-venv/bin/activate`) and execute `pip install -r demo-requirements.txt` to install the dependencies.

Next, let us set-up LakeFS. You can do this by executing the `docker run` command given here [lakeFS quickstart](https://docs.lakefs.io/quickstart/launch.html) in a terminal of your choice. Open a browser and navigate to the lakeFS instance (per default`localhost:8000`). Authenticate with the credentials given in the terminal where you executed the docker container. As an email, you can enter anything, we won't need it in this example. Proceed to create an empty repository. You may call it `weather`.


We will also install the CLI of lakeFS, `lakectl`. Then lakeFS-spec can automatically handle authentication. To do so, open a terminal of your choice and `brew install lakefs`. Then use `lakectl config`. You find the authentication information in the terminal window where you started the LakeFS Docker container. 

Note: for this to work, you need the `pyyaml` package which is not a default dependency of LakeFS-spec. We already installed via the `demo-requirements.txt`. In you own projects you need to add the dependency manually, if you want to use the LakeCTL authentication.

In [1]:
REPO_NAME = 'weather'

Now it's time to get some data. We will use the [Open-Meteo api](https://open-meteo.com/), where we can pull weather data from an API for free (as long as we are non-commercial) and without an API-token.  

For trainig we get the full data of the 2010s from Munich (where I am writing this right now ;) ) 

In [2]:
!curl -o './data/weather-2010s.json' 'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2010-01-01&end_date=2019-12-31&hourly=temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 6297k    0 6297k    0     0  1195k      0 --:--:--  0:00:05 --:--:-- 1312k7k      0 --:--:--  0:00:01 --:--:-- 1108k


The data is in JSON format so we need to wrangle the data a bit to make it usable. But first we will save it into our lakeFS instance. We will create a new branch, `transform-raw-data`.

In [3]:
from lakefs_spec import LakeFSFileSystem

NEW_BRANCH_NAME = 'transform-raw-data'


fs = LakeFSFileSystem()
fs.put('./data/weather-2010s.json',  f'{REPO_NAME}/{NEW_BRANCH_NAME}/weather-2010.json')



Created new branch 'transform-raw-data' from branch 'main'.


Now, on LakeFS in your browser, can change the branch to `transform-raw-data` and see the saved file. However, the change is not yet committed. 
While you can do that manually via the uncommitted changes tab in Lakefs, we will take another route to commit. 

We can also do commit changes programmatically. To achieve this, we register a hook. The hook needs to have the signature `(client, context) -> None`. Where the `client` is the respective LakeFS client. The context object contains information about the corresponding resource. 



In [4]:
from lakefs_client.client import LakeFSClient
from lakefs_spec.client_helpers import commit
from lakefs_spec.hooks import FSEvent, HookContext

#Define the commit hook
def commit_on_put(client: LakeFSClient, ctx:HookContext) -> None:
    commit_message = f"Add file {ctx.resource}"
    print(f"Attempting Commit: {commit_message}")
    commit(client, repository=ctx.repository, branch = ctx.ref, message=commit_message)
    

#Register the commit hook to be executed after a PUT_FILE event
fs.register_hook(FSEvent.PUT_FILE, commit_on_put)


When you execute the next cell however, you will see a message indicating that the upload of the resource has been skipped because the file is uploaded to LakeFS already (checksums match). This is useful when we work with large files to reduce the amount of network traffic. Nonetheless, in this specific situation the `PUT` is not executed and neither is our commit hook. 


In [5]:
fs.put('./data/weather-2010s.json',  f'{REPO_NAME}/{NEW_BRANCH_NAME}/weather-2010.json')

Skipping upload of resource '/Users/maxmynter/Desktop/appliedAI/lakefs/spec/demos/data/weather-2010s.json' to remote path 'weather/transform-raw-data/weather-2010.json': Resource 'weather/transform-raw-data/weather-2010.json' exists and checksums match.


We can circumvent this by disabling checking the checksums on a specific put operation. We do this by passing `precheck=False` to the `PUT` operation.

In [6]:
fs.put('./data/weather-2010s.json',  f'{REPO_NAME}/{NEW_BRANCH_NAME}/weather-2010.json', precheck=False)

Attempting Commit: Add file weather-2010.json


Now let's transform the data for our use case. We put the transformation into a function such that we can reuse it later

In this notebook, we follow a simple toy example to predict whether it is raining at the same time tomorrow given weather data from right now. 

However, we will skip a lot of possible feature engineering etc. in order to focus on the application of LakeFS and LakeFS-spec. 

In [7]:
import pandas as pd
import numpy as np
import json 

def transform_json_weather_data(filepath):
    f = open(filepath)
    data = json.load(f)
    f.close()

    df = pd.DataFrame.from_dict(data["hourly"])
    df.time = pd.to_datetime(df.time)
    df['is_raining'] = df.rain > 0
    df['is_raining_in_1_day'] = df.is_raining.shift(24)
    df = df.dropna()
    return df
    
df = transform_json_weather_data('./data/weather-2010s.json')
df.head(5)


,time,temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m,is_raining,is_raining_in_1_day
24,2010-01-02 00:00:00,-3.0,88,0.0,1004.0,999.2,100,54,100,70,8.3,17.3,18,31,False,False
25,2010-01-02 01:00:00,-3.2,89,0.0,1004.5,999.7,100,37,98,92,9.1,18.6,9,22,False,False
26,2010-01-02 02:00:00,-3.4,89,0.0,1005.2,1000.4,100,24,98,77,10.1,20.3,2,13,False,False
27,2010-01-02 03:00:00,-3.5,89,0.0,1005.6,1000.8,93,8,98,89,11.2,21.7,4,12,False,False
28,2010-01-02 04:00:00,-3.7,90,0.0,1006.2,1001.4,94,6,100,95,11.2,21.8,358,9,False,False


Let us now save this data as a csv into the main branch. The `.to_csv` method calls a `put` operation behind the scenes, our commit hook is called and the file committed. You can verify the saving worked in your LakeFS GUI in the browser when looking at the Comits and uncommitted changes tabs of the main branch. 

In [8]:
df.to_csv(f'lakefs://{REPO_NAME}/main/weather_2010s.csv')

Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file weather_2010s.csv


We will now do a train test split.

In [9]:
import sklearn.model_selection

model_data=df.drop('time', axis=1)

train, test = sklearn.model_selection.train_test_split(model_data)

We save these train and test datasets into a new `training` branch. If, as in this case, the branch does not yet exist, it is implicitly created. You can steer this behaviour via the `create_branch_ok` flag when initializing the `LakeFSFileSystem`. The flag defaults to `True`.  

In [10]:
TRAINING_BRANCH = 'training'
train.to_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/train_weather.csv')
test.to_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/test_weather.csv')


Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Created new branch 'training' from branch 'main'.
Attempting Commit: Add file train_weather.csv
Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file test_weather.csv


The implicit branch creation is a convenient way to create new branches programmatically. However, one drawback is that typos might result in new branch creation. If you want these to throw errors instead, disable the implicit branch creation

We can now read these files directly from the remote LakeFS instance. (You can verify that neither the train nor the test file are saved in the `/data` directory). 

In [11]:
train = pd.read_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/train_weather.csv', index_col=0)
test = pd.read_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/test_weather.csv', index_col=0)

train.head()

,temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m,is_raining,is_raining_in_1_day
40255,20.5,83,0.0,1015.7,1011.2,44,4,51,31,8.2,11.3,322,323,False,False
26345,4.9,89,0.0,1021.7,1016.9,86,85,12,6,17.7,31.7,257,262,False,True
8914,1.7,97,0.0,1012.1,1007.3,100,95,0,99,5.7,9.4,252,274,False,True
68183,13.9,74,0.0,1013.9,1009.3,93,97,4,10,16.0,28.6,224,229,False,False
86307,6.8,96,0.0,997.1,992.5,100,100,97,96,13.7,24.4,243,248,False,False


We now proceed to train a random forest classifier and evaluate it on the test set. 

In [12]:
from sklearn.ensemble import RandomForestClassifier

dependent_variable = 'is_raining_in_1_day'

model = RandomForestClassifier()
x_train, y_train = train.drop(dependent_variable, axis=1), train[dependent_variable].astype(bool)
x_test, y_test = test.drop(dependent_variable, axis=1), test[dependent_variable].astype(bool)

model.fit(x_train, y_train)

test_acc = model.score(x_test, y_test)

print(f"Test accuracy: {round(test_acc, 4) * 100 } %")

Test accuracy: 88.92 %


Until now, we only have used data from the 2010s. Lets get additional 2020s data, transform it and save it to LakeFS. 

In [13]:
!curl -o './data/weather-2020s.json' 'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2020-01-01&end_date=2023-08-31&hourly=temperature_2m,relativehumidity_2m,rain,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m'

new_data = transform_json_weather_data('./data/weather-2020s.json')
new_data.to_csv(f'lakefs://{REPO_NAME}/main/weather_2020s.csv')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2308k    0 2308k    0     0  1092k      0 --:--:--  0:00:02 --:--:-- 1094k
Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file weather_2020s.csv


Let's test how well our model performs on 2020s data.

First, we drop the `time` column such that we have the same variables as during the fit in the data. 

In [14]:
new_data = new_data.drop('time', axis=1)

In [15]:
acc_2020s = model.score(new_data.drop(dependent_variable, axis=1), new_data[dependent_variable].astype(bool))

print(f"Test accuracy: {round(acc_2020s, 4) * 100 } %")

Test accuracy: 84.54 %


We have an accuracy similar to the one we had on the 2020s data. Yet, it makes sense to utilize this data for training as well. We will create a concatenated dataframe and perform a new train test split. 

However, this gives rise to the problem, that we now have multiple models which will perfom differently on different datasets. For example, if someone takes the model we are about to train and evaluates it on the data from the 2020s the accuracy will probably be higher. But it will be higher because of data leakage. We are going to use some of the data points in the 2020s data to train. 

To circumvent this issue (or at least enable the traceability and reproducibility) we should save the `ref` of the specific datasets. `ref` can be the branch we are pulling the file from LakeFS from.  


We are going to implement versioning with the commit ids now.

First we create the new train test split and save it in the training branch.


In [16]:
df_train = pd.read_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/train_weather.csv', index_col=0)
df_test = pd.read_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/test_weather.csv', index_col=0)

full_data = pd.concat([new_data, df_train, df_test])

train_df, test_df = sklearn.model_selection.train_test_split(full_data, test_size=0.9)

train_df.to_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/train_weather.csv')
test_df.to_csv(f'lakefs://{REPO_NAME}/{TRAINING_BRANCH}/test_weather.csv')

Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file train_weather.csv
Calling open() in write mode results in unbuffered file uploads, because the lakeFS Python client does not support multipart uploads. Note that uploading large files unbuffered can have performance implications.
Attempting Commit: Add file test_weather.csv


This now concatenated the old data, created a new train test split and overwrote the files. This presents problems with respect to strict versioning. When we get the data using only the branch name and filename we get the latest commit. 

Lets use explicit versioning and get the explicit commit. Therefore go into the LakeFS GUI, select the training branch and choose the "Commits" tab. 

You should see the latest two Commits `Add file test_weather.csv` and `Add file train_weather.csv`.

Copy the ID to your clipboard and paste it below. (They should look something like this `be9e4c17be128bd86a082e9c5eb63135160699edd135b8b6eb78180d070b31a1`).

In [18]:
test_commit_id  = ''
train_commit_id = ''

We can now get the specific dataset versions irrespective of subsequent changes to the files on the branches

In [19]:
train = pd.read_csv(f"lakefs://{REPO_NAME}/{train_commit_id}/train_weather.csv", index_col=0)
test = pd.read_csv(f"lakefs://{REPO_NAME}/{test_commit_id}/test_weather.csv", index_col=0)

Let's train the model

In [25]:
model.fit(train.drop(dependent_variable, axis=1), train[dependent_variable].astype(bool))

test_acc = model.score(test.drop(dependent_variable, axis=1), test[dependent_variable].astype(bool))

print(f"Test accuracy: {round(test_acc, 4) * 100 } %")

Test accuracy: 86.82 %


We're done. We have a model trained on the new data. 

Now we can save the commit ids as well as the model and accuracy metrics into an experiment tracking tool of our choice. With this, we achieve reproducible experiments and have a clear trail on what data input resulted in which model weights with the resulting hyperparameter results.  

If in the future we decide to train another model or switch the data up or invest more time in feature engineering, we can always come back to the current state to study the model performance and draw insights that might help us down the road. 